# Query Agent Demo

## Connect to the Weaviate Cloud instance

> Reminder: Weaviate Agents are only available for Weaviate Cloud instances.

Connect to your Weaviate instance, using credentials from the Weaviate Cloud console. Here, they are loaded from the `.env` file.

In [1]:
from dotenv import load_dotenv
import weaviate
import os

load_dotenv()

weaviate_url = os.getenv("WEAVIATE_URL")
weaviate_api_key = os.getenv("WEAVIATE_API_KEY")

client = weaviate.connect_to_weaviate_cloud(
    cluster_url=weaviate_url,
    auth_credentials=weaviate_api_key,
)

assert client.is_ready()

## Add data

We add two datasets here, one with books and another with movies. The datasets are loaded from the Hugging Face Hub, and they are pre-vectorized using `Snowflake/snowflake-arctic-embed-l-v2.0`. 

### Load data & inspect it briefly

In [2]:
from datasets import load_dataset

movies_dataset = load_dataset("jphwang/weaviate-demos", "movies", split="train", streaming=True)
books_dataset = load_dataset("weaviate/agents", "query-agent-books", split="train", streaming=True)

In [3]:
for d in [movies_dataset, books_dataset]:
    print(f"Dataset: {d.config_name}")
    counter = 0
    for o in d:
        if counter >= 5:
            break
        print(o)
        counter += 1

Dataset: movies
{'properties': {'release_date': Timestamp('2021-12-15 00:00:00'), 'title': 'Spider-Man: No Way Home', 'overview': 'Peter Parker is unmasked and no longer able to separate his normal life from the high-stakes of being a super-hero. When he asks for help from Doctor Strange the stakes become even more dangerous, forcing him to discover what it truly means to be Spider-Man.', 'popularity': 5083.9541015625, 'vote_count': 8940, 'vote_average': 8.300000190734863, 'original_language': 'en', 'genre': 'Action, Adventure, Science Fiction', 'poster_url': 'https://image.tmdb.org/t/p/original/1g0dhYtq4irTY1GPXvft6k4YLjm.jpg'}, 'vector': [0.006379283033311367, 0.0007750422228127718, -0.011245766654610634, -0.047450534999370575, 0.0037151696160435677, -0.008632986806333065, 0.06163017451763153, 0.028453655540943146, -0.04843633621931076, -0.02299284003674984, 0.001112703001126647, -0.012430387549102306, -0.010701630264520645, -0.05831466242671013, -0.05151097849011421, 0.0872640982270

### Prepare the Collections

Here we create collections and add the objects. 

> ❗️ The `QueryAgent` uses the descriptions of collections and properties to decide which ones to use when solving queries, and to access more information about properties. You can experiment with changing these descriptions, providing more detail, and more. It's good practice to provide property descriptions too.

In [4]:
# ONLY run this if you want to delete the existing collection & data
client.collections.delete(["Movie", "Book"])

In [5]:
from weaviate.classes.config import Configure, Property, DataType

if not client.collections.exists("Movie"):
    client.collections.create(
        "Movie",
        description="A dataset that lists movies, their ratings, original language etc..",
        properties=[
            Property(
                name="title",
                data_type=DataType.TEXT,
                description="The title of the movie",
            ),
            Property(
                name="release_year",
                data_type=DataType.INT,
                description="The release year of the movie",
            ),
            Property(
                name="overview",
                data_type=DataType.TEXT,
                description="Short description of the movie",
            ),
            Property(
                name="genres",
                data_type=DataType.TEXT_ARRAY,
                description="The genres of the movie, in an array format",
            ),
            Property(
                name="vote_average",
                data_type=DataType.NUMBER,
                description="The average user rating of the movie; range is 0-10",
            ),
            Property(
                name="vote_count",
                data_type=DataType.INT,
                description="The number of user votes for the movie",
            ),
            Property(
                name="popularity",
                data_type=DataType.NUMBER,
                description="Calculated popularity of the movie by weighing multiple factors; range is 0-100",
            ),
            Property(
                name="poster_url",
                data_type=DataType.TEXT,
                description="A TMDB URL of the movie poster image",
            ),
            Property(
                name="original_language",
                data_type=DataType.TEXT,
                description="A two-letter code (e.g. 'en') representing the original language of the movie",
            ),
        ],
        vectorizer_config=[
            Configure.NamedVectors.text2vec_weaviate(
                name="default",
                source_properties=["title", "description"],
                model="Snowflake/snowflake-arctic-embed-l-v2.0"
            )
        ],
    )

Now, create a collection for books - with "Book" as the collection name, "title", "author", and "description" as text properties, and "genres" as a text array property.

Use the same vectorizer model as above, vectorizing the "title" and "description" properties.

In [ ]:
if not client.collections.exists("Book"):
    client.collections.create(
        "Book",
        description="A dataset that lists books, their author, description and genres",
        properties=[
            # Create properties for 'title', 'author', 'description' (use text types), and 'genres' (text array)
            # BEGIN_SOLUTION
            Property(name="title", data_type=DataType.TEXT, description="Title of the book"),
            Property(name="author", data_type=DataType.TEXT, description="Author of the book"),
            Property(name="description", data_type=DataType.TEXT, description="Description of the book"),
            Property(name="genres", data_type=DataType.TEXT_ARRAY, description="Genres of the book"),
            # END_SOLUTION
        ],
        vectorizer_config=[
            Configure.NamedVectors.text2vec_weaviate(
                name="default",
                source_properties=["title", "description"],
                model="Snowflake/snowflake-arctic-embed-l-v2.0"
            ),
        ],
    )


Import data

In [ ]:
from tqdm import tqdm
from weaviate.util import generate_uuid5

movies = client.collections.get("Movie")

with movies.batch.fixed_size(batch_size=100) as batch:
    for item in tqdm(movies_dataset):
        obj = item["properties"]

        # Convert release_date to release_year
        obj["release_year"] = obj["release_date"].year
        obj.pop("release_date")

        # Add object to batch for import
        batch.add_object(
            properties=item["properties"],
            uuid=generate_uuid5(item["properties"]["title"]),
            vector={"default": item["vector"]},
        )

# Check for any failed objects during import
if movies.batch.failed_objects:
    print(f"{len(movies.batch.failed_objects)} objects failed during import:")
    for failed in movies.batch.failed_objects[:3]:
        print(failed.message)

Now, try importing the books dataset into the "Book" collection. You can use the same pattern as above. Here, you shouldn't need to convert any data types, as all properties are text or text array properties.

In [ ]:

from tqdm import tqdm
from weaviate.util import generate_uuid5

books = client.collections.get("Book")

# Batch import the books data
# BEGIN_SOLUTION
with books.batch.fixed_size(batch_size=100) as batch:
    for item in tqdm(books_dataset):
        batch.add_object(
            properties=item["properties"],
            uuid=generate_uuid5(item["properties"]["title"]),
            vector={"default": item["vector"]},
        )
# END_SOLUTION

if books.batch.failed_objects:
    print(f"{len(books.batch.failed_objects)} objects failed during import:")
    for failed in books.batch.failed_objects[:3]:
        print(failed.message)

## Use Weaviate Query Agent

### Set up the query agent

In [ ]:
from weaviate.agents.query import QueryAgent
from weaviate.agents.classes import QueryAgentCollectionConfig

agent = QueryAgent(client=client, collections=[
    QueryAgentCollectionConfig(name="Movie", target_vector="default"),
    QueryAgentCollectionConfig(name="Book", target_vector="default"),
])

The `QueryAgent` will determine wither a given query is a regular search query (vector search), whether it requires aggregations, or both.

In [ ]:
response = agent.run("What are some good fantasy films that involve elves?")

In [ ]:
print(response.final_answer)

In [ ]:
response.display()

You can ask follow-up question as shown below, by providing the previous response as `context`.

In [ ]:
new_response = agent.run(
    "And what are some books with a similar vibe to these films?",
    context=response,
)

Print the final answer to the query:

In [ ]:
# BEGIN_SOLUTION
print(new_response.final_answer)
# END_SOLUTION

Display the entire response, as shown before

In [ ]:
# BEGIN_SOLUTION
new_response.display()
# END_SOLUTION

Try your own query here, based on the example above.

In [ ]:
# BEGIN_SOLUTION
response = agent.run("Do we have any spycraft related novels?")
print(response.final_answer)
# END_SOLUTION

The agent will select the appropriate collection and query type, based on the user's query and the available data.

In [ ]:
response = agent.run("Which author has the most books listed in our collection?")
print(response.final_answer)

In [ ]:
response = agent.run("What genres are the most common for this author?", context=response)
print(response.final_answer)

In [ ]:
response.display()

#### Considerations - data structure & query limitations

Note that the agent can only form queries based on the data structure and the collections available. Consider whether the following is easily executable by the agent.

In [ ]:
response = agent.run("What movies do we have in the collection that are based on this author's books?", context=response)
print(response.final_answer)

In [ ]:
response.display()

This above query may be a little tricky, as the current data doesn't provide an easy way to evaluate whether a movie is based on a book, and if so, who the original author is. 

### Search multiple collections at once

In [ ]:
multi_collection_query = """
I'm interested in movies and book that are based on European historical events, modern or ancient.
Can you recommend any good ones?
"""

Run the above query:

In [ ]:
# BEGIN_SOLUTION
response = agent.run(multi_collection_query)
# END_SOLUTION

Display the entire response

In [ ]:
# BEGIN_SOLUTION
response.display()
# END_SOLUTION

### Streaming output

Streaming output is available for the `QueryAgent`. You can use it to get partial results as they are generated, rather than waiting for the entire response to be ready.

These intermediate results can provide a more interactive experience for the user, as they can see the progress in real-time, and get a sense of how the agent is approaching the query.

In [ ]:
from weaviate.agents.classes import ProgressMessage, StreamedTokens

for output in agent.stream(
    "What are some good fantasy films that involve elves?",
    # Setting this to false will skip ProgressMessages, and only stream
    # the StreamedTokens / the final QueryAgentResponse
    include_progress=True  # Default is True
):
    if isinstance(output, ProgressMessage):
        # The message is a human-readable string, structured info available in output.details
        print(output.message)
    elif isinstance(output, StreamedTokens):
        # The delta is a string containing the next chunk of the final answer
        print(output.delta, end='', flush=True)
    else:
        # This is the final response, as returned by QueryAgent.run()
        output.display()

Can you find all Harry Potter books in our database? Try it, with streaming output enabled.

In [ ]:
from weaviate.agents.classes import ProgressMessage, StreamedTokens

# BEGIN_SOLUTION
for output in agent.stream(
    "How many Harry Potter books are in our database? Please list them all",
    include_progress=True
):
    if isinstance(output, ProgressMessage):
        print(output.message)
    elif isinstance(output, StreamedTokens):
        print(output.delta)
    else:
        output.display()
# END_SOLUTION

Remember to close the client and clean up the connections :) 

In [ ]:
client.close()